---
title: GSB - S544 Lab 3
author: Karissa Mohr
format:
  html:
    embed-resources: true
    toc: true
    toc-depth: 3
echo: true
theme: zephyr
---

Load Data Set

In [1]:
import pandas as pd

xmas = pd.read_csv("https://www.dropbox.com/scl/fi/qxaslqqp5p08i1650rpc4/xmas.csv?rlkey=erdxi7jbh7pqf9fh4lv4cayp5&dl=1")
xmas

,Day,Day.in.Words,Gift.Item,Verb,Adjective,Location
0,1,first,partridge,NaN,NaN,in a pear tree
1,2,second,dove,NaN,turtle,NaN
2,3,third,hen,NaN,french,NaN
3,4,fourth,bird,NaN,calling,NaN
4,5,fifth,ring,NaN,golden,NaN
5,6,sixth,goose,a-laying,NaN,NaN
6,7,seventh,swan,a-swimming,NaN,NaN
7,8,eighth,maid,a-milking,NaN,NaN
8,9,ninth,lady,dancing,NaN,NaN
9,10,tenth,lord,a-leaping,NaN,NaN


Function 1: pluralize_gift()

In [2]:
def pluralize_gift(gift):
    """
    Returns plural of a noun

    Parameters
    ----------
    gift: str
        A noun

    Returns
    -------
        Plural version of the noun
    """
    # Step 1
    if not isinstance(gift, str):
        return gift

    # Step 2
    if "oo" in gift:
        gift = gift.replace("oo", "ee")
    elif gift.endswith("y") and gift[-2] not in "aeiou":
        gift = gift[:-1] + "ies"
    else:
        gift = gift + "s"

    return gift

Test your function

In [4]:

print(pluralize_gift("goose"))

geese


In [6]:
xmas["Gift.Item"].map(pluralize_gift)

,Gift.Item
0,partridges
1,doves
2,hens
3,birds
4,rings
5,geese
6,swans
7,maids
8,ladies
9,lords


Function 2: make_phrase()

In [15]:
def make_phrase(num, num_word, item, verb, adjective, location):
  """
  Builds the phrase.

  Parameters
  ----------
  num : int
      The day
  num_word : str
      The word version of the number
  item : str
      The gift item
  verb : str
       Action
  adjective : str
      Adjective
  location : str
      Location

  Returns
  -------
  str
      The completed phrase
  """
  # Step 1: Replace NAs with blank strings
  if isinstance(verb, pd.Series):
      verb = verb.fillna("")
      adjective = adjective.fillna("")
      location = location.fillna("")
  else:
      verb = "" if pd.isna(verb) else verb
      adjective = "" if pd.isna(adjective) else adjective
      location = "" if pd.isna(location) else location

  ## Step 2: If the day number is larger than 1, pluralize the gift item
  ### Hint: call the function you created above!
  if num > 1:
      item = pluralize_gift(item)

  ## Step 3: Figure out if a gift item starts with a vowel
  vowels = "aeiou"
  starts_with_vowel = item[0].lower() in vowels

  ## Step 4: For the first day, use "a"/"an" based on vowel
  ## If it's not the first day, use just the number word (e.g. "ten lords")
  if num == 1:
      num_word = "an" if starts_with_vowel else "a"

  ## Step 5: Put all of the pieces together into one string and return!
  phrase_parts = [num_word, adjective, item, verb, location]
  phrase = " ".join([p for p in phrase_parts if p.strip() != ""]).strip()

  return phrase

The Day.in.Words variable isn’t quite what you want! You want 12 to say "twelve" not "twelfth". Consider using .map() and a dictionary to make a column of the words you need.

In [16]:
num_to_word = {
    1: "one",
    2: "two",
    3: "three",
    4: "four",
    5: "five",
    6: "six",
    7: "seven",
    8: "eight",
    9: "nine",
    10: "ten",
    11: "eleven",
    12: "twelve"
}

xmas["Number.of.Days"] = xmas["Day"].map(num_to_word)

xmas[["Day", "Day.in.Words", "Number.of.Days"]]


,Day,Day.in.Words,Number.of.Days
0,1,first,one
1,2,second,two
2,3,third,three
3,4,fourth,four
4,5,fifth,five
5,6,sixth,six
6,7,seventh,seven
7,8,eighth,eight
8,9,ninth,nine
9,10,tenth,ten


Test Your Function

In [17]:
print(make_phrase(
    num=10,
    num_word="ten",
    item="lord",
    verb="a-leaping",
    adjective="",
    location=""
))


ten lords a-leaping


In [18]:
print(make_phrase(
    num=1,
    num_word="one",
    item="partridge",
    verb="",
    adjective="",
    location="in a pear tree"
))

a partridge in a pear tree


In [19]:
xmas["Full.Phrase"] = xmas.apply(
    lambda row: make_phrase(
        row["Day"],
        row["Number.of.Days"],
        row["Gift.Item"],
        row["Verb"],
        row["Adjective"],
        row["Location"]
    ), axis=1
)

# Display the results
xmas[["Day", "Full.Phrase"]]


,Day,Full.Phrase
0,1,a partridge in a pear tree
1,2,two turtle doves
2,3,three french hens
3,4,four calling birds
4,5,five golden rings
5,6,six geese a-laying
6,7,seven swans a-swimming
7,8,eight maids a-milking
8,9,nine ladies dancing
9,10,ten lords a-leaping


Function 3: sing_day()

In [20]:
def sing_day(dataset, num, phrase_col):
  """
  Sings the verse for a specific day of Christmas

  Parameters
  ----------
  dataset : pd.DataFrame
      The full dataset containing the gift phrases
  num : int
      The day number to sing
  phrase_col : str
      The name of the column that contains the gift phrases

  Returns
  -------
  str
      The full verse for that day of Christmas
  """

  # Step 1: Setup the intro line
  num_to_ordinal = {
      1: "first",
      2: "second",
      3: "third",
      4: "fourth",
      5: "fifth",
      6: "sixth",
      7: "seventh",
      8: "eighth",
      9: "ninth",
      10: "tenth",
      11: "eleventh",
      12: "twelfth"
  }

  num_word = num_to_ordinal[num]
  intro = "On the " + num_word + " day of Christmas, my true love gave to me:"

  # Step 2: Sing the gift phrases
  gifts = ""
  for i in range(num, 0, -1):
    gift_phrase = dataset.loc[i - 1, phrase_col]
    if i == 1 and num > 1:
        gifts += "and " + gift_phrase + "."
    elif i == 1 and num == 1:
        gifts += gift_phrase + "."
    else:
        gifts += gift_phrase + ",\n"

  # Step 3: Put it all together and return
  verse = intro + "\n" + gifts
  return verse

Test your function


In [21]:
print(sing_day(xmas, 1, "Full.Phrase"))

On the first day of Christmas, my true love gave to me:
a partridge in a pear tree.


Use Your Functions!

In [22]:
# Sing the 12 Days od Christmas Song

for day in range(1, 13):
    print(sing_day(xmas, day, "Full.Phrase"))
    print()


On the first day of Christmas, my true love gave to me:
a partridge in a pear tree.

On the second day of Christmas, my true love gave to me:
two turtle doves,
and a partridge in a pear tree.

On the third day of Christmas, my true love gave to me:
three french hens,
two turtle doves,
and a partridge in a pear tree.

On the fourth day of Christmas, my true love gave to me:
four calling birds,
three french hens,
two turtle doves,
and a partridge in a pear tree.

On the fifth day of Christmas, my true love gave to me:
five golden rings,
four calling birds,
three french hens,
two turtle doves,
and a partridge in a pear tree.

On the sixth day of Christmas, my true love gave to me:
six geese a-laying,
five golden rings,
four calling birds,
three french hens,
two turtle doves,
and a partridge in a pear tree.

On the seventh day of Christmas, my true love gave to me:
seven swans a-swimming,
six geese a-laying,
five golden rings,
four calling birds,
three french hens,
two turtle doves,
and a 

In [23]:
xmas2 = pd.read_csv("https://www.dropbox.com/scl/fi/p9x9k8xwuzs9rhp582vfy/xmas_2.csv?rlkey=kvc3j3lmyn4opcidsrhcmrof1&dl=1")

In [24]:
num_to_word = {
    1: "one",
    2: "two",
    3: "three",
    4: "four",
    5: "five",
    6: "six",
    7: "seven",
    8: "eight",
    9: "nine",
    10: "ten",
    11: "eleven",
    12: "twelve"
}

xmas2["Number.of.Days"] = xmas2["Day"].map(num_to_word)
xmas2[["Day", "Day.in.Words", "Number.of.Days"]]


,Day,Day.in.Words,Number.of.Days
0,1,first,one
1,2,second,two
2,3,third,three
3,4,fourth,four
4,5,fifth,five
5,6,sixth,six
6,7,seventh,seven
7,8,eighth,eight
8,9,ninth,nine
9,10,tenth,ten


In [26]:
xmas2["Full.Phrase"] = xmas2.apply(
    lambda row: make_phrase(
        row["Day"],
        row["Number.of.Days"],
        row["Gift.Item"],
        row["Verb"],
        row["Adjective"],
        row["Location"]
    ),
    axis=1
)

xmas2[["Day", "Full.Phrase"]]

,Day,Full.Phrase
0,1,an email from Cal Poly
1,2,two meal points
2,3,three lost pens
3,4,four course reviews
4,5,five practice exams
5,6,six graders grading
6,7,seven seniors stressing
7,8,eight moms a-calling
8,9,nine parties bumping
9,10,ten loads of laundry


In [27]:
for day in range(1, 13):
    print(sing_day(xmas2, day, "Full.Phrase"))
    print()


On the first day of Christmas, my true love gave to me:
an email from Cal Poly.

On the second day of Christmas, my true love gave to me:
two meal points,
and an email from Cal Poly.

On the third day of Christmas, my true love gave to me:
three lost pens,
two meal points,
and an email from Cal Poly.

On the fourth day of Christmas, my true love gave to me:
four course reviews,
three lost pens,
two meal points,
and an email from Cal Poly.

On the fifth day of Christmas, my true love gave to me:
five practice exams,
four course reviews,
three lost pens,
two meal points,
and an email from Cal Poly.

On the sixth day of Christmas, my true love gave to me:
six graders grading,
five practice exams,
four course reviews,
three lost pens,
two meal points,
and an email from Cal Poly.

On the seventh day of Christmas, my true love gave to me:
seven seniors stressing,
six graders grading,
five practice exams,
four course reviews,
three lost pens,
two meal points,
and an email from Cal Poly.

On t